# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#import logging
import os
import json
#import csv

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-146203
aml-quickstarts-146203
southcentralus
6971f5ac-8af1-446e-8034-05acea24681f


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-heart-failure-prediction'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

experiment

Name,Workspace,Report Page,Docs Page
automl-heart-failure-prediction,quick-starts-ws-146203,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-31T07:52:32.557000+00:00', 'errors': None, 'creationTime': '2021-05-31T07:09:50.344955+00:00', 'modifiedTime': '2021-05-31T07:10:35.835717+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction DataSet from UCI"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure-prediction,AutoML_840efe9b-43cc-450c-a802-44143d3b2876,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure-prediction,AutoML_840efe9b-43cc-450c-a802-44143d3b2876,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_840efe9b-43cc-450c-a802-44143d3b2876',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-31T08:17:24.768675Z',
 'endTimeUtc': '2021-05-31T08:43:31.182099Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-failure-prediction","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-146203","workspace_name":"quick-starts-ws-146203","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, fitted_model = remote_run.get_output()
best_metrics = best_run.get_metrics()
print("Best run id : {}".format(best_run.id),
      "Accuracy : {}".format(best_metrics['accuracy']),
      "Best metrics : {}".format(best_metrics),
      "Best model : {}".format(fitted_model), sep = '\n'
     )

Best run id : AutoML_840efe9b-43cc-450c-a802-44143d3b2876_36
Accuracy : 0.8831034482758622
Best metrics : {'AUC_micro': 0.9232964724534286, 'norm_macro_recall': 0.7122691404070122, 'weighted_accuracy': 0.9001914543127907, 'balanced_accuracy': 0.856134570203506, 'f1_score_micro': 0.8831034482758622, 'precision_score_macro': 0.8914661727161727, 'recall_score_micro': 0.8831034482758622, 'average_precision_score_macro': 0.9077931242201183, 'precision_score_micro': 0.8831034482758622, 'f1_score_macro': 0.8613521484905784, 'precision_score_weighted': 0.8969990067403859, 'AUC_weighted': 0.9179789370219575, 'recall_score_weighted': 0.8831034482758622, 'log_loss': 0.3632027859095686, 'accuracy': 0.8831034482758622, 'matthews_correlation': 0.744732814920156, 'average_precision_score_micro': 0.9260725297971717, 'recall_score_macro': 0.856134570203506, 'average_precision_score_weighted': 0.9302726214520177, 'AUC_macro': 0.9179789370219575, 'f1_score_weighted': 0.8797289003131847, 'confusion_matrix

In [12]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(
    estimators=[('5', Pipeline(memory=None,
             steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')),
                    ('xgboostclassifier',
                     XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=10, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
        dataset_samples=299,
        dataset_features=12,
        dataset_classes=2,
        dataset_num_cate...
        iteration_timeout_mode=0,
        iteration_timeout_param=None,
        feature_column_names=None,
        label_column_name=None,
        weight_column_name=None,
        cv_split_column_names=None,
        enable_streaming=None,
        timeseries_param_dict=None,
        gpu_training_param_dict={'processing_unit_type': 'cpu'}
    ), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=1.25, subsample=0.5, tree_method='auto'))],
             verbose=False)), ('13', Pipel

In [13]:
#TODO: Save the best model
model = best_run.register_model(model_name = "automl-heart-failure-best-model", model_path = './outputs/model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-146203', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-146203'), name=automl-heart-failure-best-model, id=automl-heart-failure-best-model:1, version=1, tags={}, properties={})


In [14]:
for model in Model.list(ws):
    print(model.name, 'Version: ', model.version)

automl-heart-failure-best-model Version:  1
hyperd-heart-failure-best-model Version:  2
hyperd-heart-failure-best-model Version:  1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
best_run.download_file('/outputs/model.pkl', os.path.join('./model','automl-heart-failure-best-model'))
for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [16]:
model = best_run.register_model(model_name = 'automl-heart-failure-best-model', model_path = './outputs/model.pkl')

In [17]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-environment', file_path = 'conda_env.yml')

In [18]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [19]:
inference_config = InferenceConfig(entry_script = 'score.py', environment = environment)

In [20]:
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights = True)

In [21]:
webservice = Model.deploy(workspace = ws, name = 'heart-failure-service', models=[model], inference_config = inference_config, deployment_config = aci_config, overwrite = True)

In [22]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-146203', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-146203'), name=heart-failure-service, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [23]:
webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-31 08:51:25+00:00 Creating Container Registry if not exists.
2021-05-31 08:51:35+00:00 Registering the environment.
2021-05-31 08:51:37+00:00 Building image..
2021-05-31 09:05:43+00:00 Generating deployment configuration.
2021-05-31 09:05:45+00:00 Submitting deployment to compute.
2021-05-31 09:05:48+00:00 Checking the status of deployment heart-failure-service..
2021-05-31 09:09:46+00:00 Checking the status of inference endpoint heart-failure-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [26]:
test_data = df.sample(5)
test_label = test_data.pop('DEATH_EVENT')
data_json = test_data.to_dict(orient='records')
data = json.dumps({'data' : data_json})
data

'{"data": [{"age": 72.0, "anaemia": 1, "creatinine_phosphokinase": 110, "diabetes": 0, "ejection_fraction": 25, "high_blood_pressure": 0, "platelets": 274000.0, "serum_creatinine": 1.0, "serum_sodium": 140, "sex": 1, "smoking": 1, "time": 65}, {"age": 56.0, "anaemia": 1, "creatinine_phosphokinase": 135, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 133000.0, "serum_creatinine": 1.7, "serum_sodium": 140, "sex": 1, "smoking": 0, "time": 244}, {"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 14, "high_blood_pressure": 0, "platelets": 166000.0, "serum_creatinine": 0.8, "serum_sodium": 127, "sex": 1, "smoking": 0, "time": 14}, {"age": 64.0, "anaemia": 0, "creatinine_phosphokinase": 143, "diabetes": 0, "ejection_fraction": 25, "high_blood_pressure": 0, "platelets": 246000.0, "serum_creatinine": 2.4, "serum_sodium": 135, "sex": 1, "smoking": 0, "time": 214}, {"age": 70.0, "anaemia": 0, "creatinine_phosphokinase":

In [32]:
data = {"data": [{"age": 72.0, 
                   "anaemia": 1, 
                   "creatinine_phosphokinase": 110, 
                   "diabetes": 0, "ejection_fraction": 25, 
                   "high_blood_pressure": 0, 
                   "platelets": 274000.0, 
                   "serum_creatinine": 1.0, 
                   "serum_sodium": 140, 
                   "sex": 1, 
                   "smoking": 1, 
                   "time": 65}, 
                  {"age": 56.0, 
                   "anaemia": 1, 
                   "creatinine_phosphokinase": 135, 
                   "diabetes": 1, 
                   "ejection_fraction": 38, 
                   "high_blood_pressure": 0, 
                   "platelets": 133000.0, 
                   "serum_creatinine": 1.7, 
                   "serum_sodium": 140, 
                   "sex": 1, 
                   "smoking": 0, 
                   "time": 244}, 
                 ]}
request = json.dumps(data)

In [35]:
output = webservice.run(request)
output

'{"result": [1, 0]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [36]:
webservice.get_logs()

'2021-05-31T09:09:42,397552100+00:00 - iot-server/run \n2021-05-31T09:09:42,401681200+00:00 - rsyslog/run \n2021-05-31T09:09:42,441937200+00:00 - gunicorn/run \n2021-05-31T09:09:42,536962600+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_146c4c7d40342c77f7b6edb9f83abd51/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-05-31T09:09:42,981717900+00:00 - iot-server/finish 1 0\n2021-05-31T09:09:42,987744600+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (69)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 98\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-05-31 09:09:46,137 | root | INFO | Starting up app insights client\n2021-05-31 09:09:46,143 | root | INFO | Starting up request id generato

In [37]:
webservice.delete()